In [47]:
import pandas as pd
import numpy as np

import warnings
warnings = "ignore_warnings" 

In [48]:
df =  pd.read_csv(r"E:/documents/python practice/new/tvs.csv")

In [49]:
df.head(2)

,_id,created_by,first_air_date,genres,id,in_production,last_air_date,name,number_of_episodes,number_of_seasons,...,original_language,original_name,overview,popularity,poster_path,production_companies,status,tagline,vote_average,vote_count
0,6457d26c7b901e08b141869d,[],1996-10-09,"[{""id"":35,""name"":""Comedy"",""_id"":{""$oid"":""6457d...",13,True,2022-02-16,How do you like Wednesday?,568.0,4,...,ja,水曜どうでしょう,How do you like Wednesday? was a Japanese tele...,21.207,/i2MwS6U0XzD8ad6aS3HiTNKz8ov.jpg,[],Returning Series,NaN,9.200,2
1,6457d26c7b901e08b141867a,"[{""id"":19303,""name"":""Kevin Smith"",""_id"":{""$oid...",2000-05-31,"[{""id"":16,""name"":""Animation"",""_id"":{""$oid"":""64...",2,False,2002-12-22,Clerks,6.0,1,...,en,Clerks,The continuing adventures of store clerks Dant...,10.620,/xunXvzFlkf1GGgMkCySA9CCFumB.jpg,"[{""id"":1558,""name"":""Touchstone Television"",""lo...",Canceled,NaN,6.897,78


In [50]:
df.columns

Index(['_id', 'created_by', 'first_air_date', 'genres', 'id', 'in_production',
       'last_air_date', 'name', 'number_of_episodes', 'number_of_seasons',
       'origin_country', 'original_language', 'original_name', 'overview',
       'popularity', 'poster_path', 'production_companies', 'status',
       'tagline', 'vote_average', 'vote_count'],
      dtype='object')

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150802 entries, 0 to 150801
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   _id                   150802 non-null  object 
 1   created_by            150802 non-null  object 
 2   first_air_date        119932 non-null  object 
 3   genres                150802 non-null  object 
 4   id                    150802 non-null  int64  
 5   in_production         150802 non-null  bool   
 6   last_air_date         122108 non-null  object 
 7   name                  150796 non-null  object 
 8   number_of_episodes    150168 non-null  float64
 9   number_of_seasons     150802 non-null  int64  
 10  origin_country        150802 non-null  object 
 11  original_language     150802 non-null  object 
 12  original_name         150796 non-null  object 
 13  overview              84623 non-null   object 
 14  popularity            150802 non-null  float64
 15  

- First of all i'm gonna delte some columns which is not import for our model
- i want just some columns like genres overview, tagline, name, and maybe production_com
- but before that i wanna i wanna drop every single null values in a column which is import for our model

***Actully i have tried to make a model with 28k dimensions but i was getting lots of error and the size of the data was aboutt 6 GB so i reduce the size of the data and i just took the most import and famous tv shows from the data and applyy the model on it and now it's size about 550 Mbs:***

In [52]:
columns = ['_id', 'first_air_date', 'in_production',
       'last_air_date', 'number_of_episodes', 'number_of_seasons',
       'original_name',
       'popularity', 'status',
      'vote_average', 'tagline']

df = df.drop(columns = columns)

- **There is some empty list where is nothing in these columns so let's find out how much theyyy are in each columns**

In [53]:
df['created_by'].map(lambda x: np.nan if x == [] or x== '[]' else x).count()

32601

In [54]:
df.shape

(150802, 10)

In [55]:
for col in df.columns:
    empt_list = df[col].map(lambda x: np.nan if x == [] or x== '[]' else x).count()
    print(col,empt_list)

created_by 32601
genres 88497
id 150802
name 150796
origin_country 118147
original_language 150802
overview 84623
poster_path 94927
production_companies 52049
vote_count 150802


In [56]:
# This will transfer both to nan
df = df.map(lambda x: np.nan if x == [] or x == "[]" else x)

In [57]:
df.isnull().sum()

created_by              118201
genres                   62305
id                           0
name                         6
origin_country           32655
original_language            0
overview                 66179
poster_path              55875
production_companies     98753
vote_count                   0
dtype: int64

In [58]:
# i want to drop all the null rows espcially if it's in the following columns because the rest i will delete even though if it's \
    # not empty because only we need textiual data for our model
col_list = ['created_by', 'name', 'genres', 'origin_country', 'original_language', 'overview', 'poster_path', 'production_companies']
df = df.dropna(subset= col_list)

In [59]:
# Now i want to make it more smaller i mean less than 10k so i would perform filter on vote_count collumn actually \
    # i want to drop the values if it's less than 2 mean 200 vote_count
df = df[df['vote_count'] >2]

In [60]:
df.shape

(8536, 10)

In [61]:
df.isnull().sum()

created_by              0
genres                  0
id                      0
name                    0
origin_country          0
original_language       0
overview                0
poster_path             0
production_companies    0
vote_count              0
dtype: int64

In [62]:
# now i wanna drop null values
df.dropna(inplace=True)
df = df.drop(columns= 'vote_count')

## Function to fetch the name from the values

In [63]:
import ast 

In [64]:
import json

def fetch_genres(obj):
    name_list = []  
    try:
        # Parse the JSON string
        data = json.loads(obj)
        # Extract names
        for i in data:
            name_list.append(i['name'])
    except (json.JSONDecodeError, KeyError, TypeError):
        return None
    return name_list

In [65]:
df['genres'].apply(fetch_genres)

1                                       [Animation, Comedy]
3                                                    [Kids]
5                                                  [Comedy]
6         [Kids, Animation, Action & Adventure, Sci-Fi &...
8                                     [Drama, Soap, Comedy]
                                ...                        
148489                                              [Drama]
148490                                       [Drama, Crime]
148809                                              [Drama]
149054                                    [Comedy, Mystery]
149653                      [Drama, Family, Crime, Mystery]
Name: genres, Length: 8536, dtype: object

In [66]:
# I'm going to save the clean column
df['genres'] = df['genres'].apply(fetch_genres)

**Now let's apply the same function on production_companies if it's work or no?**

In [67]:
# apply the fetch_json on companies column
df['production_companies'] = df['production_companies'].apply(fetch_genres)

In [68]:
df['created_by'] = df['created_by'].apply(fetch_genres)

In [69]:
df.head()

,created_by,genres,id,name,origin_country,original_language,overview,poster_path,production_companies
1,"[Kevin Smith, Scott Mosier, David Mandel]","[Animation, Comedy]",2,Clerks,"[""US""]",en,The continuing adventures of store clerks Dant...,/xunXvzFlkf1GGgMkCySA9CCFumB.jpg,"[Touchstone Television, View Askew Productions..."
3,[Fred Rogers],[Kids],15,Mister Rogers' Neighborhood,"[""US""]",en,Mister Rogers' Neighborhood is an American chi...,/4Gz46uJg6MFaM4xeblbVNSxCyTr.jpg,"[WQED, Small World Enterprises, Family Communi..."
5,[Bill Bixby],[Comedy],18,W*A*L*T*E*R,"[""US""]",en,W*A*L*T*E*R is a pilot for a spin-off of M*A*S...,/fwSw4fl08MVl5w6Q9lVUGIFkZQ2.jpg,[20th Century Fox Television]
6,"[George Lucas, Ben Burtt, Clive A. Smith, Raym...","[Kids, Animation, Action & Adventure, Sci-Fi &...",25,Star Wars: Droids,"[""CA"",""US""]",en,An animated television series that features th...,/b3gsZSTauRBJDJvWCbD9oVAsFe1.jpg,"[Lucasfilm Animation, Lucasfilm Ltd.]"
8,"[Gail Parent, Norman Lear, Ann Marcus, Jerry A...","[Drama, Soap, Comedy]",29,"Mary Hartman, Mary Hartman","[""US""]",en,"In the fictional town of Fernwood, Ohio, subur...",/O1bcYEBW8v7JOMNVpYOBTL6PjL.jpg,[TAT Communications Company]


In [70]:
# To remove the quotation from the list
# df['origin_country'] = df['origin_country'].apply(lambda x: ast.literal_eval(x))
df['origin_country'] = df['origin_country'].apply(lambda x: x.strip('[]').replace('"', ''))

In [71]:
# i want this column to be in a list form too
df['overview'] = df['overview'].apply(lambda x: x.split())

## Removing spaces

In [72]:
# No spaces between words
df['genres'].map(lambda x: [i.replace(" ", "") for i in x])

1                                       [Animation, Comedy]
3                                                    [Kids]
5                                                  [Comedy]
6         [Kids, Animation, Action&Adventure, Sci-Fi&Fan...
8                                     [Drama, Soap, Comedy]
                                ...                        
148489                                              [Drama]
148490                                       [Drama, Crime]
148809                                              [Drama]
149054                                    [Comedy, Mystery]
149653                      [Drama, Family, Crime, Mystery]
Name: genres, Length: 8536, dtype: object

In [73]:
df.head()

,created_by,genres,id,name,origin_country,original_language,overview,poster_path,production_companies
1,"[Kevin Smith, Scott Mosier, David Mandel]","[Animation, Comedy]",2,Clerks,US,en,"[The, continuing, adventures, of, store, clerk...",/xunXvzFlkf1GGgMkCySA9CCFumB.jpg,"[Touchstone Television, View Askew Productions..."
3,[Fred Rogers],[Kids],15,Mister Rogers' Neighborhood,US,en,"[Mister, Rogers', Neighborhood, is, an, Americ...",/4Gz46uJg6MFaM4xeblbVNSxCyTr.jpg,"[WQED, Small World Enterprises, Family Communi..."
5,[Bill Bixby],[Comedy],18,W*A*L*T*E*R,US,en,"[W*A*L*T*E*R, is, a, pilot, for, a, spin-off, ...",/fwSw4fl08MVl5w6Q9lVUGIFkZQ2.jpg,[20th Century Fox Television]
6,"[George Lucas, Ben Burtt, Clive A. Smith, Raym...","[Kids, Animation, Action & Adventure, Sci-Fi &...",25,Star Wars: Droids,"CA,US",en,"[An, animated, television, series, that, featu...",/b3gsZSTauRBJDJvWCbD9oVAsFe1.jpg,"[Lucasfilm Animation, Lucasfilm Ltd.]"
8,"[Gail Parent, Norman Lear, Ann Marcus, Jerry A...","[Drama, Soap, Comedy]",29,"Mary Hartman, Mary Hartman",US,en,"[In, the, fictional, town, of, Fernwood,, Ohio...",/O1bcYEBW8v7JOMNVpYOBTL6PjL.jpg,[TAT Communications Company]


In [74]:
# No spaces between words between each word
df['production_companies'] = df['production_companies'].map(lambda x: [i.replace(" ", "") for i in x])
df['genres'] = df['genres'].map(lambda x: [i.replace(" ", "") for i in x])
df['created_by'] = df['created_by'].map(lambda x: [i.replace(" ", "") for i in x])


In [75]:
print(type(df['created_by'].iloc[1]))  # Check type of the first value
print(type(df['genres'].iloc[1]))      # Check type of the first value
print(type(df['origin_country'].iloc[1]))  # Check type of the first value
print(type(df['original_language'].iloc[1]))  
print(type(df['overview'].iloc[1]))  
print(type(df['production_companies'].iloc[1]))   


<class 'list'>
<class 'list'>
<class 'str'>
<class 'str'>
<class 'list'>
<class 'list'>


In [76]:
 # removing the quotes from the original language and change it to list
df['original_language'].apply(lambda x: x.split(", "))

1         [en]
3         [en]
5         [en]
6         [en]
8         [en]
          ... 
148489    [es]
148490    [nl]
148809    [tr]
149054    [hi]
149653    [zh]
Name: original_language, Length: 8536, dtype: object

In [77]:
# removing the quotes from the original country and change it to list same with language
df['origin_country'] = df['origin_country'].apply(lambda x: x.replace('"', '').split())
df['original_language'] = df['original_language'].apply(lambda x: x.split(", "))

In [78]:
# i make a new column to combine everything together
df['tags'] = df['genres'] + df['origin_country'] + df['origin_country'] + df['overview'] + df['production_companies'] + df['created_by']

In [79]:
df = df.reset_index(drop=True)

In [80]:
# new_df is now ready for more process 
new_df = df[['id', 'name', 'tags', 'poster_path']]

In [81]:
# Now removing the list from the tags column
new_df['tags'] = new_df['tags'].map(lambda x: " ".join(x))

C:\Users\JASIM\AppData\Local\Temp\ipykernel_15516\2263816752.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].map(lambda x: " ".join(x))


In [82]:
# Change the text into lower case
new_df['tags'] = new_df['tags'].map(lambda x: x.lower())

C:\Users\JASIM\AppData\Local\Temp\ipykernel_15516\184896880.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].map(lambda x: x.lower())


## Stemming

In [83]:
import nltk
from nltk.stem.porter import PorterStemmer
pr = PorterStemmer()

In [84]:
# the function to apply the stemmer to the features tags
def stemer(text):
    list = []

    for i in text.split():
        list.append(pr.stem(i))
    return " ".join(list)

In [85]:
new_df['tags'].apply(stemer)

0       anim comedi us us the continu adventur of stor...
1       kid us us mister rogers' neighborhood is an am...
2       comedi us us w*a*l*t*e*r is a pilot for a spin...
3       kid anim action&adventur sci-fi&fantasi ca,u c...
4       drama soap comedi us us in the fiction town of...
                              ...                        
8531    drama ar ar no one can and should live without...
8532    drama crime be be when a prodig son send hi fa...
8533    drama tr tr the stori revolv around the love t...
8534    comedi mysteri in in a rebelli vampir with a b...
8535    drama famili crime mysteri cn cn the stori rev...
Name: tags, Length: 8536, dtype: object

In [86]:
new_df['tags'] = new_df['tags'].apply(stemer)

C:\Users\JASIM\AppData\Local\Temp\ipykernel_15516\71712778.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stemer)


## Count Vectorizer

In [87]:
# couunt vectorizer will change the words to numbers into numpyy array
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=4000, stop_words='english')

In [88]:
# savving in variable 
vectors = cv.fit_transform(new_df['tags']).toarray()

In [89]:
cv.get_feature_names_out()[70:100]


array(['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2016', '2017', '2022', '20th', '20thcenturyfoxtelevis',
       '20thtelevis', '20thtelevisionanim', '21', '21st', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '30s', '31', '32', '35'],
      dtype=object)

In [90]:
vectors.shape

(8536, 4000)

## Cosine Similarity content based

okayy now we are gonna apply the technique after all that how much the each tv shows or similar to ech other  is far from each other in dimension we can use two of them cosign similirty or uclidient but becuaae we have hight dimension so cosign simility is mroe better 

In [91]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_score = cosine_similarity(vectors)

In [92]:
new_df[new_df['name'] == 'Vikings'].index[0]

2958

In [93]:
similarity_score.shape

(8536, 8536)

In [94]:
data_sizes = similarity_score.nbytes/ ( 1024**2)

print(f'Size of the similarity matrix: {data_sizes:.2f} MB')

Size of the similarity matrix: 555.90 MB


## Function for tv Similarity 

In [95]:
# getting index by name
new_df[new_df['name'] == "Vikings"].index[0]

2958

In [96]:
sorted(list(enumerate(similarity_score[1])), reverse=True, key= lambda x: x[1])[1:6]

[(966, 0.4262183570801332),
 (150, 0.41316814406864993),
 (1925, 0.40459194096600426),
 (851, 0.4004906973834362),
 (830, 0.39977519855091365)]

In [97]:
def show_similarity(series):
    series_index = new_df[new_df['name'] == series].index[0]
    similarity = similarity_score[series_index]
    five_similar = sorted(list(enumerate(similarity)), reverse=True, key= lambda x: x[1])[1:6]
    for i, score in five_similar:
        name = new_df.loc[i]['name']
        print(name)

In [98]:
show_similarity('Vikings')

The Empire
Baa Baa Black Sheep
Slugterra
The Legend of El Cid
Big Wolf on Campus


i make a function where we are gonna get the 5 or 6 similiar series to the specific tv shows so 
first of all i filter based on seriees name and get their index 
first of all i get one values in similarity and then i use enumerate function because it will keep the index and the similirtyyy both other wise if directly sort the values it will disturb the index of all series so thats why then we just sort in decending order based on score 

### Actually i want to reduce the data becasue it's too larger

- **i wanted to make a larger tv shows recommened system but i got an error because it was too larger**

- **I'm gonna use cosine similarity and reduce the data size**

 - actually i'll reduce the data size by the vote count to get the most famous tv shows so it will be better because most of the people might know about this kind of tv shows so i'll edit the data from the beginning

In [ ]:
data_size = similarity_score.nbytes/ ( 1024**2)
print(f'Size of the similarity matrix: {data_size:.2f} MB')

Size of the similarity matrix: 6466.87 MB


**Now i'll apply the process again in some cell**

i would remove the data if the vote count is less than 20 butt important first of all i'll remove the data if import columns is null in it you might think the vote count is too low but actually it's not because even the most famous shows in this dataset like this according to my own research  the data vote count is 100 times less so if you wanna get real resurlt you have to do this (vote_count * 100)

- *i want to drop all the null rows espcially if it's in the following columns because the rest i will delete even though if it's not empty because only we need textiual data for our model*

## Saving the model and data

***actually the data is so big so it's hard to save with pickle or joblic etc so i use h5py after my research but still i was getting an issues in streamlit so i reduce the size of the data and then i make it 550 mbs but before it was more than 6 GB***

In [ ]:
import pickle

with open('similarity_file.pkl', 'wb') as file:
    pickle.dump(similarity_score, file)

In [ ]:
import pickle

with open('data.pkl', 'wb') as file:
    pickle.dump(new_df, file)
